## Final Project Submission

Please fill out:
* Student name: Tabitha Berum
* Student pace: full time
* Scheduled project review date/time: 22, March 2024
* Instructor name: Diana Mongina
* Blog post URL: N/A


# Overview
The project's goal is to create an analysis using information on movies, including production budgets, sales and the percentage of viewers that like each film, gathered from many websites. This analysis's outcome will make it easier to select the greatest film to produce financially for the future Microsoft Studios.

# Business Understanding
Microsoft Studios wants to produce original video content, but they have no idea how to go about doing so. The goal of this analysis is to explore what types of films are doing the best and will most likely generate revenue post production.
This will be done in the following steps:
    1. Defining key factors to evaluate a film's profitability using production budget, ratings or gross revenue.
    2. Examining profitability of different movie attributes like release date, genre, director, length of the movie and  popularity.
    3. Coming up with a step by step guide of choosing the best movie financially using the 5 W's to gether with the attributes mentioned above.

# Data Understanding

This analysis will focus on three datasets:
    -Box Office Movies, this data entails the gross sales made on movies produced.
    
    -The Movie DB, this is data entails the ratings (votes fro audiences) on movies based on the release dates and genre of the produced movies.

    -The Numbers, this gives us an insight of the cost of the entire process of making a movie from production, release date and the gross revenue that was made from the movies.

##      Data Preparation

In [55]:
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [56]:
# Import library that enables us to read zip files
import zipfile 

#Extract IMDB SQL .db file
# with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:

#    zipObj.extractall('./zippedData/')

In [57]:
#Read tables using sqlite3
import sqlite3
conn = sqlite3.connect('ZippedData/im.db')

In [58]:
# Check for table existence
#cursor = conn.cursor()
#cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
#tables = cursor.fetchall()

# Print table names
#for table in tables:
  #  print(table[0])

# Close database connection
#conn.close()


In [59]:
# Loading data from all datasets
movie_data = pd.read_csv('./zippeddata/rt.movie_info.tsv.gz', sep="\t")
reviews_data = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep="\t", encoding= 'unicode_escape')
gross_data = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
tmdb_data = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
budgets_data = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')



After loading the data , we are going to explore each dataset to know if there's any anomalies. We begin with the Rotten Tomatoes dataset

## Rotten Tomatoes

In [60]:
#Fetch first 5 indexes of each dataset
display(
        gross_data.head(),
        tmdb_data.head(),
        budgets_data.head()
        )

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [61]:
# Checking for the information in the gross_data and tmdb_data datasets.
display(
    gross_data.info(),
    tmdb_data.info()
    )


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date     

None

None

In [62]:
# Checking for all the information in the budgets_data dataset
budgets_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [63]:
# Checking for missing values
display(
    gross_data.isnull().sum(),
    tmdb_data.isnull().sum(),
    budgets_data.isnull().sum()
)

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [64]:
# Drop rows with missing values
gross_data.dropna(inplace=True)

In [65]:
# Checking for values that have duplicates in the datasets
display(
    gross_data.duplicated().sum(),
    tmdb_data.duplicated().sum(),
    budgets_data.duplicated().sum()
    )

0

0

0